In [8]:
import requests
import pandas as pd

stationen: https://planner.dbcargo.com/stations.json

- erstmal alle Stationen darüber bekommen
    - nach german filtern
- erweiterte ids herausfinden
    - stationen ohne routingcode haben meistens keine verbindungen (have yet to find some)
    - stationen ohne routingcode kann man rausschmeißen
- übriggebliebenen durchsuchen, quadratische laufzeit maybe
- es gibt stationen die haben routingcode xxx-00, 
    - andere stationen haben xxx-yy, diese sind maybe unterBahnhöfe von den -00 dingern, vllt kann man die auch skippen


In [294]:
def get_stations():
    headers = {}
    search_url = f"https://planner.dbcargo.com/stations.json"
    params = {}
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
        d = response.json()
    return d


In [295]:
def get_departing(id, country = 80):
    base = "https://planner.dbcargo.com/nwd/commercialLocations"
    params = {
        "countryNumber": country,
        "locationNumber" : id,
        "locationType" : "Dispatch",
        "mappingType" : "Partner"
    }
    response = requests.get(url = base, params=params)
    d = None
    if response.status_code == 200:
        return response
    return None

In [296]:
def dataframe_response(data):
    def unpack_single_element_lists(d):
        if isinstance(d, dict):
            return {k: unpack_single_element_lists(v) for k, v in d.items()}
        elif isinstance(d, list) and len(d) > 0:
            return unpack_single_element_lists(d[0])  # Nimmt das einzige Element
        return d  # Andernfalls unverändert zurückgeben

    data2 = unpack_single_element_lists(data)

    # JSON normalisieren
    normalized_data = pd.json_normalize(data2)
    return normalized_data




In [297]:
payload = {
    "arrivalLocationId" : 22922245,
    "departureLocationId": 22920962,
    "dispatchLocationId" : 22920961,
    "earliestTimeOfDay" : 0,
    "latestTimeOfDay" : 86399,
    "mappingType": "Partner",
    "receivingLocationId" : 22929561,
    "routingCode" : "630-00",
    "scenarioHash": "2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed",
    "searchMode": "Departure",
    "weekdays": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

}

In [298]:
base = "https://planner.dbcargo.com/nwd/timetableQuery"

response = requests.post(url=base,json= payload)
response

<Response [200]>

In [299]:
rep = response.json()

In [300]:
rep

{'scenarioInformation': {'scenarioHash': '2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed',
  'validFrom': '2025-03-03T00:00:00+01:00',
  'validUntil': '2025-04-08T00:00:00+02:00',
  'referenceWeekBegin': '2025-03-03T00:00:00+01:00',
  'referenceWeekEnd': '2025-03-09T00:00:00+01:00',
  'timetableTimestamp': '2025-02-28T13:11:23.170156+01:00'},
 'connectionRequestParameterSet': {'dispatchLocationId': 22920961,
  'receivingLocationId': 22929561,
  'departureLocationId': 22920962,
  'arrivalLocationId': 22922245,
  'routingCode': '630-00',
  'searchMode': 'Departure',
  'mappingType': 'Partner',
  'weekdays': ['Monday',
   'Tuesday',
   'Wednesday',
   'Thursday',
   'Friday',
   'Saturday',
   'Sunday'],
  'earliestTimeOfDay': 0,
  'latestTimeOfDay': 86399},
 'connections': [{'compositionEndWeekdays': ['Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday'],
   'compositionEndTimeOfDay': 61200,
   'decompositionEndWeekdays': ['Tuesday',
    'Wednesday',
 

In [301]:
stations = get_stations()
df = pd.DataFrame(stations)


In [302]:
df

,id,name,country
0,000141,Sörnäinen,10
1,000406,Riihimäki,10
2,000471,Hämeenlinna,10
3,000737,Hanko,10
4,000752,Lappohja,10
...,...,...,...
21303,exclude,Pinghu Nan,China
21304,exclude,Pinghu,
21305,exclude,Sungang,
21306,exclude,Shekou,


In [303]:
german = df[df["country"] == 80]
german = german.drop("country",axis = 1).reset_index().drop("index",axis = 1)

In [304]:
german

,id,name
0,010033,Hamburg-Who Ct Burchardkai Ctb
1,010066,Kudensee
2,010116,Maschen MVZ
3,010124,Maschen Rbf Werk Y-Schad
4,010660,Hamburg-Billwerder Ubf
...,...,...
2209,999953,MALUS
2210,999961,KULANZ
2211,999979,LOGISTIK
2212,999987,CZ BLN SERVICE ABEGL


In [305]:
import time
#  this will take long, like 70 minutes for all german stations
def get_consumer_stations(stations):
    results = []
    for k,row in stations.iterrows():
        data = get_departing(row["id"])
        print(row["id"])
        print(k)

        if data is None:
            break

        print(data.json())
        counter = 0
        # time.sleep(1.5 ** counter)
        while "scenarioHash" not in data.json().keys():
            time.sleep(1.5 ** counter)
            data = get_departing(row["id"])
            counter += 1

        print(data.json())

        df = dataframe_response(data.json())
        print(f"length : {len(df.columns)}")
        results.append(df)
    return results

results = []
# results = get_consumer_stations(german)


In [307]:
# testdf = pd.concat(results)


In [308]:
# testdf.to_pickle("company_locations.pkl")

In [309]:
new_df = pd.read_pickle("company_locations.pkl")

In [310]:
new_df = new_df.reset_index().drop("index",axis = 1)
new_df

,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,locations.productionLocationAndRoutingCodes.productionLocation.locationNumber,locations.productionLocationAndRoutingCodes.productionLocation.name,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,010819,HAMBURG-WALTERSHOF,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
1,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361.0,80,010066,KUDENSEE,...,22920413.0,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
2,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,011759,MASCHEN RBF,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,011759,MASCHEN RBF,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,010660,HAMBURG-BILLWERDER,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2210,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2211,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2212,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [311]:
# combined = pd.concat([german,new_df],axis=1)
# combined.to_pickle("complete.pkl")

In [312]:
# combined.to_json("complete.json",orient="records")
# combined.to_csv("complete.csv",index = False)
# combined

combined = pd.read_pickle("complete.pkl")

In [313]:
new = combined.dropna(subset= ["locations.productionLocationAndRoutingCodes.routingCodes"])
new

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,010033,Hamburg-Who Ct Burchardkai Ctb,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
1,010066,Kudensee,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361.0,80,...,22920413.0,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
2,010116,Maschen MVZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,010124,Maschen Rbf Werk Y-Schad,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,010660,Hamburg-Billwerder Ubf,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,893651,Dingolfing PA Scholz,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00893651,None,MADS,22915095.0,80,...,22915096.0,80,893651,DINGOLFING PA SCHOLZ,48.642839,12.491984,DINGOLFING PA SCHOLZ,1.0,NaN,NaN
2201,894725,Bülstringen Florastrasse,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00894725,None,LFLO,22915092.0,80,...,22915093.0,80,894725,BUELSTRINGEN FLORAST,52.311128,11.333092,BUELSTRINGEN FLORAST,1.0,NaN,NaN
2202,897926,Dabel,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00897926,None,WDL,22915088.0,80,...,22915089.0,80,897926,DABEL,53.671404,11.893348,DABEL,2.0,NaN,NaN
2203,899385,Krefeld Industriebahn,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00899385,None,KKRK,22919136.0,80,...,22930374.0,80,899385,KREFELD INDUSTRIEBHN,51.342592,6.591727,KREFELD INDUSTRIEBHN,1.0,NaN,NaN


In [285]:
df_cleaned = new.drop_duplicates(subset=["locations.productionLocationAndRoutingCodes.routingCodes"], keep="last")

In [286]:
df_cleaned_routing = df_cleaned.drop_duplicates(subset= ["locations.productionLocationAndRoutingCodes.productionLocation.latitude"],keep="last")

In [287]:
df_cleaned_routing.columns

Index(['id', 'name', 'scenarioHash', 'locations.companyLocationNumberOwner',
       'locations.type', 'locations.tafTsiCode',
       'locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber',
       'locations.productionLocationAndRoutingCodes.productionLocation.code',
       'locations.productionLocationAndRoutingCodes.productionLocation.id',
       'locations.productionLocationAndRoutingCodes.productionLocation.countryNumber',
       'locations.productionLocationAndRoutingCodes.productionLocation.locationNumber',
       'locations.productionLocationAndRoutingCodes.productionLocation.name',
       'locations.productionLocationAndRoutingCodes.productionLocation.latitude',
       'locations.productionLocationAndRoutingCodes.productionLocation.longitude',
       'locations.productionLocationAndRoutingCodes.productionLocation.objectKeyAlpha',
       'locations.productionLocationAndRoutingCodes.productionLocation.objectKeySequence',
       'locations.product

In [288]:
import plotly.express as px

In [289]:
fig = px.scatter(df_cleaned_routing, y="locations.productionLocationAndRoutingCodes.productionLocation.latitude",
                  x="locations.productionLocationAndRoutingCodes.productionLocation.longitude", 
                  title="Interaktiver 2D-Scatterplot",
                  hover_data=["name"])
fig.update_layout(
    xaxis=dict(scaleanchor="y"),  # X- und Y-Skala verknüpfen
    yaxis=dict(scaleanchor="x"),
    width=800,  # Quadrat erzwingen
    height=800
)
fig.update_traces(
    hovertemplate='%{customdata}<extra></extra>'  # Zeigt nur den Namen an
)

# Custom-Daten für den Namen hinzufügen
fig.update_traces(customdata=df_cleaned_routing["name"])

fig.show()

In [290]:
df_duplicates = combined[combined.duplicated(subset=["locations.productionLocationAndRoutingCodes.productionLocation.latitude"], keep=False)]
df_duplicates

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,010033,Hamburg-Who Ct Burchardkai Ctb,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
2,010116,Maschen MVZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,010124,Maschen Rbf Werk Y-Schad,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,010660,Hamburg-Billwerder Ubf,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
5,010777,Cuxhaven Fischereihafen Cuxport-Terminal,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010777,None,ACV,22920465.0,80,...,22929272.0,80,010777,CUXHAVEN FISCH CUX T,53.849509,8.726545,CUXHAVEN FISCH CUX T,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,999953,MALUS,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2210,999961,KULANZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2211,999979,LOGISTIK,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2212,999987,CZ BLN SERVICE ABEGL,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [233]:
df_duplicates2 =  df_duplicates[df_duplicates.duplicated(subset=["locations.productionLocationAndRoutingCodes.productionLocation.latitude"], keep=False)]

In [ ]:
df_duplicates

(713, 27)

In [236]:
df_duplicates2.shape

(713, 27)